## Setup Inicial

Configuração do ambiente e definição de variáveis do Unity Catalog.
Importação de bibliotecas necessárias para transformações de dados.

In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, trim, upper, lower, initcap, current_timestamp, 
    lit, coalesce, when, regexp_replace, length, row_number, to_timestamp,levenshtein,length
)
from pyspark.sql.types import IntegerType, StringType, TimestampType, StructType, StructField
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
from pyspark.sql.functions import encode, decode, col, desc, unix_timestamp, round, try_to_timestamp

In [0]:
catalog_name = "atendimento_catalog"
bronze_db_name = "bronze"
silver_db_name = "silver"

In [0]:
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {silver_db_name}")

## Funções Utilitárias

Funções auxiliares para validação, normalização e transformação de dados.
Incluem tratamento de timezone para horário de Brasília e remoção de acentos.

In [0]:
def safe_table_exists(spark, full_name: str) -> bool:
    try:
        spark.table(full_name)
        return True
    except AnalysisException:
        return False
    except Exception:
        return False

def safe_col(df, name):
    return col(name) if name in df.columns else lit(None).cast(StringType())

def safe_cast_int(col_expr):
    return when(
        col_expr.isNotNull() & (trim(col_expr).cast(StringType()) != ""),
        F.regexp_replace(trim(col_expr).cast(StringType()), r'[^\d]', '').cast(IntegerType())
    ).otherwise(None)

def remove_accents_udf(text_col):
    return F.translate(
        text_col,
        "áàãâäéèêëíìîïóòõôöúùûüçñÁÀÃÂÄÉÈÊËÍÌÎÏÓÒÕÔÖÚÙÛÜÇÑ",
        "aaaaaeeeeiiiiooooouuuucnAAAAAEEEEIIIIOOOOOUUUUCN"
    )

def normalize_text(col_expr):
    return trim(regexp_replace(regexp_replace(col_expr, r'\s+', ' '), r'[^\x20-\x7E\u00C0-\u00FF]', ''))

def clean_brazil_timestamp(col_expr):
    cleaned = regexp_replace(col_expr, r'[�]', ' ')
    cleaned = regexp_replace(cleaned, r'\bàs\b|\bas\b|\bàs\b', ' ')
    cleaned = regexp_replace(cleaned, r'\s+', ' ')
    cleaned = trim(cleaned)
    return cleaned

def parse_to_brasilia_timezone(col_expr):
    cleaned = clean_brazil_timestamp(col_expr)
    parsed_naive = to_timestamp(cleaned, 'dd/MM/yyyy HH:mm:ss')
    parsed_utc = F.to_utc_timestamp(parsed_naive, 'America/Sao_Paulo')
    parsed_brasilia = F.from_utc_timestamp(parsed_utc, 'America/Sao_Paulo')
    return parsed_brasilia
    
def similarity(col1, col2):
    return (1 - (levenshtein(col1, col2) / F.greatest(length(col1), length(col2)))) * 100

## Processamento: ft_atendentes

Transformação da tabela de atendentes da camada Bronze para Silver.
Aplicação de normalização de nomes e validação de níveis de atendimento.

### Leitura da Tabela Bronze

Carregamento da tabela `ft_atendentes` da camada Bronze.
Validação de existência e contagem inicial de registros.

In [0]:
src_table_atendentes = f"{catalog_name}.{bronze_db_name}.ft_atendentes"
tgt_table_atendentes = f"{catalog_name}.{silver_db_name}.ft_atendentes"

if not safe_table_exists(spark, src_table_atendentes):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_atendentes}")

df_atendentes_src = spark.table(src_table_atendentes)
total_before_atendentes = df_atendentes_src.count()

print(f"Leitura: {src_table_atendentes}")
print(f"Registros Bronze: {total_before_atendentes:,}")

### Transformação e Normalização

Aplicação de transformações para padronização de tipos e formatos.
Coerção de IDs para IntegerType e normalização de nomes com remoção de acentos.

In [0]:
df_atendentes = df_atendentes_src \
    .withColumn("id_atendente_raw", safe_col(df_atendentes_src, "id_atendente")) \
    .withColumn("nome_atendente_raw", safe_col(df_atendentes_src, "nome_atendente")) \
    .withColumn("nivel_atendimento_raw", safe_col(df_atendentes_src, "nivel_atendimento"))

df_atendentes = df_atendentes.withColumn(
    "id_atendente",
    safe_cast_int(col("id_atendente_raw"))
)

df_atendentes = df_atendentes.withColumn(
    "nome_atendente_normalized",
    normalize_text(col("nome_atendente_raw"))
)

df_atendentes = df_atendentes.withColumn(
    "nome_atendente",
    initcap(remove_accents_udf(col("nome_atendente_normalized")))
)

df_atendentes = df_atendentes.withColumn(
    "nivel_atendimento",
    when(
        safe_cast_int(col("nivel_atendimento_raw")).isin([1, 2]),
        safe_cast_int(col("nivel_atendimento_raw"))
    ).otherwise(None)
)

print(f"Transformações aplicadas")


### Regras de Transformação

**Normalização de Nome:**
- Remoção de espaços extras e caracteres especiais
- Remoção de acentos (João → Joao, André → Andre)
- Aplicação de initcap (primeira letra maiúscula)

**Coerção de ID:**
- Conversão segura para IntegerType
- Remoção de caracteres não numéricos
- Valores inválidos resultam em NULL

**Validação de Nível:**
- Apenas valores 1 ou 2 são aceitos
- Valores fora do intervalo são convertidos para NULL

### Validações de Qualidade

Aplicação de regras de validação para garantir qualidade dos dados.
Remoção de registros com campos obrigatórios inválidos ou nulos.

In [0]:
null_id_atendente = df_atendentes.filter(col("id_atendente").isNull()).count()
null_nome_atendente = df_atendentes.filter(col("nome_atendente").isNull() | (trim(col("nome_atendente")) == "")).count()
null_nivel_atendimento = df_atendentes.filter(col("nivel_atendimento").isNull()).count()

print(f"Validação de campos obrigatórios:")
print(f"  - id_atendente NULL: {null_id_atendente:,}")
print(f"  - nome_atendente NULL/vazio: {null_nome_atendente:,}")
print(f"  - nivel_atendimento NULL: {null_nivel_atendimento:,}")

df_atendentes_valid = df_atendentes.filter(
    (col("id_atendente").isNotNull()) & (col("id_atendente") > 0) &
    (col("nome_atendente").isNotNull()) & (trim(col("nome_atendente")) != "") &
    (col("nivel_atendimento").isNotNull())
)

removed_by_validation_atendentes = total_before_atendentes - df_atendentes_valid.count()
print(f"\nRegistros removidos por validação: {removed_by_validation_atendentes:,}")
print(f"Registros válidos: {df_atendentes_valid.count():,}")

### Deduplicação

Remoção de registros duplicados mantendo apenas a versão mais recente.
Ordenação por ingestion_timestamp para preservar último registro ingerido.

In [0]:
dup_count_atendentes = df_atendentes_valid.groupBy("id_atendente").count().filter(col("count") > 1).count()
print(f"Duplicatas detectadas: {dup_count_atendentes:,}")

w_atendentes = Window.partitionBy("id_atendente").orderBy(
    col("ingestion_timestamp").desc_nulls_last() if "ingestion_timestamp" in df_atendentes_valid.columns
    else lit(datetime.now())
)

df_atendentes_dedup = df_atendentes_valid \
    .withColumn("rn", row_number().over(w_atendentes)) \
    .filter(col("rn") == 1) \
    .drop("rn")

removed_by_dedup_atendentes = df_atendentes_valid.count() - df_atendentes_dedup.count()
print(f"Registros removidos por deduplicação: {removed_by_dedup_atendentes:,}")

### Preparação Final

Adição de metadados de processamento e seleção de colunas finais.
Aplicação de schema com constraints.

In [0]:
df_atendentes_final = df_atendentes_dedup.withColumn("processed_timestamp", current_timestamp())

final_cols_atendentes = [
    "id_atendente",
    "nome_atendente",
    "nivel_atendimento",
    "processed_timestamp",
    "ingestion_timestamp"
]

df_atendentes_final = df_atendentes_final.select(*[c for c in final_cols_atendentes if c in df_atendentes_final.columns])

df_atendentes_typed = df_atendentes_final \
    .withColumn("id_atendente", col("id_atendente").cast(IntegerType())) \
    .withColumn("nome_atendente", col("nome_atendente").cast(StringType())) \
    .withColumn("nivel_atendimento", col("nivel_atendimento").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType())) \
    .withColumn("ingestion_timestamp", 
                col("ingestion_timestamp").cast(TimestampType()) if "ingestion_timestamp" in df_atendentes_final.columns 
                else lit(None).cast(TimestampType()))

print(f"Total de registros finais: {df_atendentes_typed.count():,}")

### Gravação na Camada Silver

Persistência da tabela transformada em formato Delta na camada Silver.
Modo overwrite com overwriteSchema para garantir schema atualizado.

In [0]:
df_atendentes_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_atendentes)

silver_table_atendentes = spark.table(tgt_table_atendentes)
final_count_atendentes = silver_table_atendentes.count()
duplicates_atendentes = silver_table_atendentes.groupBy("id_atendente").count().filter(col("count") > 1).count()

print(f"Salvo em: {tgt_table_atendentes}")
print(f"Contagem final: {final_count_atendentes:,}")
print(f"Duplicatas na Silver: {duplicates_atendentes}")

display(silver_table_atendentes.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Bronze para Silver.
Métricas de qualidade e taxa de aproveitamento dos dados.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - FT_ATENDENTES")
print("="*80)
print(f"Registros Bronze (origem): {total_before_atendentes:,}")
print(f"Registros removidos (validação): {removed_by_validation_atendentes:,}")
print(f"Registros removidos (deduplicação): {removed_by_dedup_atendentes:,}")
print(f"Registros Silver (destino): {final_count_atendentes:,}")
print(f"Taxa de aproveitamento: {(final_count_atendentes/total_before_atendentes*100):.2f}%")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: ft_chamados_hora

Transformação da tabela de chamados da camada Bronze para Silver.
Aplicação de tratamento de timezone e validação de consistência temporal.

### Leitura da Tabela Bronze

Carregamento da tabela `ft_chamados_hora` da camada Bronze.
Validação de existência e contagem inicial de registros.

In [0]:
src_table_chamados = f"{catalog_name}.{bronze_db_name}.ft_chamados_hora"
tgt_table_chamados = f"{catalog_name}.{silver_db_name}.ft_chamados_hora"

if not safe_table_exists(spark, src_table_chamados):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_chamados}")

df_chamados_src = spark.table(src_table_chamados)
total_before_chamados = df_chamados_src.count()

print(f"Leitura: {src_table_chamados}")
print(f"Registros Bronze: {total_before_chamados:,}")

### Transformação e Normalização

Aplicação de transformações para padronização de tipos e formatos.
Coerção de id_chamado para IntegerType e id_cliente como StringType.
Correção de encoding e tratamento de timezone para timestamps.

In [0]:
df_chamados = df_chamados_src \
    .withColumn("id_chamado_raw", safe_col(df_chamados_src, "ID_Chamado")) \
    .withColumn("id_cliente_raw", safe_col(df_chamados_src, "ID_Cliente")) \
    .withColumn("hora_abertura_raw", safe_col(df_chamados_src, "Hora_Abertura_Chamado")) \
    .withColumn("hora_inicio_raw", safe_col(df_chamados_src, "Hora_Inicio_Atendimento")) \
    .withColumn("hora_finalizacao_raw", safe_col(df_chamados_src, "Hora_Finalizacao_Atendimento"))

df_chamados = df_chamados.withColumn("id_chamado", safe_cast_int(col("id_chamado_raw")))
df_chamados = df_chamados.withColumn("id_cliente", trim(col("id_cliente_raw")))

print("Amostra de dados brutos:")
df_chamados.select("hora_abertura_raw", "hora_inicio_raw", "hora_finalizacao_raw").show(3, truncate=False)

### Correção de Encoding e Parsing de Timestamps

Tratamento de timestamps com encoding incorreto e conversão para horário de Brasília.
Aplicação de timezone America/Sao_Paulo com tratamento correto de DST.

In [0]:
df_chamados = df_chamados.withColumn("hora_abertura_chamado_brasilia", parse_to_brasilia_timezone(col("hora_abertura_raw")))
df_chamados = df_chamados.withColumn("hora_inicio_atendimento_brasilia", parse_to_brasilia_timezone(col("hora_inicio_raw")))
df_chamados = df_chamados.withColumn("hora_finalizacao_atendimento_brasilia", parse_to_brasilia_timezone(col("hora_finalizacao_raw")))

print("Amostra de timestamps convertidos:")
df_chamados.select(
    "hora_abertura_chamado_brasilia", 
    "hora_inicio_atendimento_brasilia", 
    "hora_finalizacao_atendimento_brasilia"
).show(5, truncate=False)

parsing_failures_abertura = df_chamados.filter(
    col("hora_abertura_raw").isNotNull() & col("hora_abertura_chamado_brasilia").isNull()
).count()
parsing_failures_inicio = df_chamados.filter(
    col("hora_inicio_raw").isNotNull() & col("hora_inicio_atendimento_brasilia").isNull()
).count()
parsing_failures_fim = df_chamados.filter(
    col("hora_finalizacao_raw").isNotNull() & col("hora_finalizacao_atendimento_brasilia").isNull()
).count()

print(f"\nFalhas de parsing:")
print(f"  - hora_abertura: {parsing_failures_abertura:,}")
print(f"  - hora_inicio: {parsing_failures_inicio:,}")
print(f"  - hora_finalizacao: {parsing_failures_fim:,}")

### Tratamento de Timezone

**Problema:** Timestamps com encoding incorreto (dd/MM/yyyy �s HH:mm:ss)

**Solução:**
- Substituição de caracteres problemáticos
- Parsing no formato brasileiro (dd/MM/yyyy HH:mm:ss)
- Interpretação como horário de Brasília (America/Sao_Paulo)
- Conversão via UTC para garantir tratamento correto de DST
- Representação final em TimestampType no fuso de Brasília

### Validações de Qualidade

Aplicação de regras de validação para garantir qualidade dos dados.
Remoção de registros com campos obrigatórios nulos ou timestamps inválidos.

In [0]:
null_id_chamado = df_chamados.filter(col("id_chamado").isNull()).count()
null_id_cliente = df_chamados.filter(col("id_cliente").isNull()).count()
null_hora_abertura = df_chamados.filter(col("hora_abertura_chamado_brasilia").isNull()).count()
null_hora_inicio = df_chamados.filter(col("hora_inicio_atendimento_brasilia").isNull()).count()
null_hora_fim = df_chamados.filter(col("hora_finalizacao_atendimento_brasilia").isNull()).count()

print(f"Validação de campos obrigatórios:")
print(f"  - id_chamado NULL: {null_id_chamado:,}")
print(f"  - id_cliente NULL: {null_id_cliente:,}")
print(f"  - hora_abertura_chamado_brasilia NULL: {null_hora_abertura:,}")
print(f"  - hora_inicio_atendimento_brasilia NULL: {null_hora_inicio:,}")
print(f"  - hora_finalizacao_atendimento_brasilia NULL: {null_hora_fim:,}")

df_chamados_valid_nulls = df_chamados.filter(
    (col("id_chamado").isNotNull()) & (col("id_chamado") > 0) &
    (col("id_cliente").isNotNull()) & (length(col("id_cliente")) > 0) &
    (col("hora_abertura_chamado_brasilia").isNotNull()) &
    (col("hora_inicio_atendimento_brasilia").isNotNull()) &
    (col("hora_finalizacao_atendimento_brasilia").isNotNull())
)

removed_by_nulls_chamados = total_before_chamados - df_chamados_valid_nulls.count()
print(f"\nRegistros removidos por campos obrigatórios inválidos: {removed_by_nulls_chamados:,}")
print(f"Registros restantes: {df_chamados_valid_nulls.count():,}")

### Validação de Consistência Temporal

Verificação da ordem cronológica dos eventos.
Descarte de registros com timestamps fora de ordem.

In [0]:
df_chamados_temporal = df_chamados_valid_nulls.withColumn(
    "valido_abertura_inicio",
    col("hora_abertura_chamado_brasilia") <= col("hora_inicio_atendimento_brasilia")
).withColumn(
    "valido_inicio_fim",
    col("hora_inicio_atendimento_brasilia") <= col("hora_finalizacao_atendimento_brasilia")
).withColumn(
    "valido_temporal",
    col("valido_abertura_inicio") & col("valido_inicio_fim")
)

invalidos_abertura_inicio = df_chamados_temporal.filter(~col("valido_abertura_inicio")).count()
invalidos_inicio_fim = df_chamados_temporal.filter(~col("valido_inicio_fim")).count()
invalidos_total = df_chamados_temporal.filter(~col("valido_temporal")).count()

print(f"Validação de consistência temporal:")
print(f"  - Abertura > Início (inválido): {invalidos_abertura_inicio:,}")
print(f"  - Início > Fim (inválido): {invalidos_inicio_fim:,}")
print(f"  - Total de registros temporalmente inválidos: {invalidos_total:,}")

if invalidos_total > 0:
    print(f"\nAmostra de registros com ordem temporal inválida:")
    df_chamados_temporal.filter(~col("valido_temporal")).select(
        "id_chamado",
        "hora_abertura_chamado_brasilia",
        "hora_inicio_atendimento_brasilia",
        "hora_finalizacao_atendimento_brasilia"
    ).show(5, truncate=False)

df_chamados_valid = df_chamados_temporal.filter(col("valido_temporal")).drop(
    "valido_abertura_inicio", "valido_inicio_fim", "valido_temporal"
)

removed_by_temporal_chamados = df_chamados_valid_nulls.count() - df_chamados_valid.count()
print(f"\nRegistros removidos por inconsistência temporal: {removed_by_temporal_chamados:,}")
print(f"Registros restantes: {df_chamados_valid.count():,}")

### Regras de Consistência Temporal

**Validações aplicadas:**
- hora_abertura_chamado <= hora_inicio_atendimento
- hora_inicio_atendimento <= hora_finalizacao_atendimento

**Ação:** Registros com timestamps fora de ordem são removidos.

### Deduplicação

Remoção de registros duplicados mantendo apenas a versão mais recente.
Ordenação por ingestion_timestamp para preservar último registro ingerido.

In [0]:
dup_count_chamados = df_chamados_valid.groupBy("id_chamado").count().filter(col("count") > 1).count()
print(f"Duplicatas detectadas: {dup_count_chamados:,}")

w_chamados = Window.partitionBy("id_chamado").orderBy(
    col("ingestion_timestamp").desc_nulls_last() if "ingestion_timestamp" in df_chamados_valid.columns
    else lit(datetime.now())
)

df_chamados_dedup = df_chamados_valid \
    .withColumn("rn", row_number().over(w_chamados)) \
    .filter(col("rn") == 1) \
    .drop("rn")

removed_by_dedup_chamados = df_chamados_valid.count() - df_chamados_dedup.count()
print(f"Registros removidos por deduplicação: {removed_by_dedup_chamados:,}")

### Preparação Final

Adição de metadados de processamento e seleção de colunas finais.
Aplicação de schema com constraints.

In [0]:
df_chamados_final = df_chamados_dedup.withColumn("processed_timestamp", current_timestamp())

final_cols_chamados = [
    "id_chamado",
    "id_cliente",
    "hora_abertura_chamado_brasilia",
    "hora_inicio_atendimento_brasilia",
    "hora_finalizacao_atendimento_brasilia",
    "processed_timestamp",
    "ingestion_timestamp"
]

df_chamados_final = df_chamados_final.select(*[c for c in final_cols_chamados if c in df_chamados_final.columns])

df_chamados_typed = df_chamados_final \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("hora_abertura_chamado_brasilia", col("hora_abertura_chamado_brasilia").cast(TimestampType())) \
    .withColumn("hora_inicio_atendimento_brasilia", col("hora_inicio_atendimento_brasilia").cast(TimestampType())) \
    .withColumn("hora_finalizacao_atendimento_brasilia", col("hora_finalizacao_atendimento_brasilia").cast(TimestampType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType())) \
    .withColumn("ingestion_timestamp", 
                col("ingestion_timestamp").cast(TimestampType()) if "ingestion_timestamp" in df_chamados_final.columns 
                else lit(None).cast(TimestampType()))

print(f"Total de registros finais: {df_chamados_typed.count():,}")

### Schema Final

**Colunas:**
- id_chamado: IntegerType, NOT NULL
- id_cliente: StringType, NOT NULL
- hora_abertura_chamado_brasilia: TimestampType, NOT NULL
- hora_inicio_atendimento_brasilia: TimestampType, NOT NULL
- hora_finalizacao_atendimento_brasilia: TimestampType, NOT NULL
- processed_timestamp: TimestampType
- ingestion_timestamp: TimestampType

**Nota:** Métricas de tempo foram removidas desta camada.

### Gravação na Camada Silver

Persistência da tabela transformada em formato Delta na camada Silver.
Modo overwrite com overwriteSchema para garantir schema atualizado.

In [0]:
df_chamados_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_chamados)

silver_table_chamados = spark.table(tgt_table_chamados)
final_count_chamados = silver_table_chamados.count()
duplicates_chamados = silver_table_chamados.groupBy("id_chamado").count().filter(col("count") > 1).count()

print(f"Salvo em: {tgt_table_chamados}")
print(f"Contagem final: {final_count_chamados:,}")
print(f"Duplicatas na Silver: {duplicates_chamados}")

display(silver_table_chamados.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Bronze para Silver.
Métricas de qualidade e taxa de aproveitamento dos dados.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - FT_CHAMADOS_HORA")
print("="*80)
print(f"Registros Bronze (origem): {total_before_chamados:,}")
print(f"Registros removidos (campos obrigatórios inválidos): {removed_by_nulls_chamados:,}")
print(f"Registros removidos (inconsistência temporal): {removed_by_temporal_chamados:,}")
print(f"Registros removidos (deduplicação): {removed_by_dedup_chamados:,}")
print(f"Registros Silver (destino): {final_count_chamados:,}")
print(f"Taxa de aproveitamento: {(final_count_chamados/total_before_chamados*100):.2f}%")
print("="*80)
print(f"\nDetalhamento de validações:")
print(f"  - Falhas de parsing (abertura): {parsing_failures_abertura:,}")
print(f"  - Falhas de parsing (início): {parsing_failures_inicio:,}")
print(f"  - Falhas de parsing (finalização): {parsing_failures_fim:,}")
print(f"  - Abertura > Início: {invalidos_abertura_inicio:,}")
print(f"  - Início > Fim: {invalidos_inicio_fim:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: motivos

Transformação da tabela de motivos da camada Bronze para Silver.
Padronização das descrições e validação de IDs.

In [0]:
src_table_motivos = f"{catalog_name}.{bronze_db_name}.ft_motivos" 
tgt_table_motivos = f"{catalog_name}.{silver_db_name}.ft_motivos"

if not safe_table_exists(spark, src_table_motivos):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_motivos}")

df_motivos_src = spark.table(src_table_motivos)
total_before_motivos = df_motivos_src.count()

print(f"Leitura: {src_table_motivos}")
print(f"Registros Bronze: {total_before_motivos:,}")

df_motivos = df_motivos_src \
    .withColumn("id_motivo_raw", safe_col(df_motivos_src, "id_motivo")) \
    .withColumn("nome_motivo_raw", safe_col(df_motivos_src, "nome_motivo")) \
    .withColumn("categoria_raw", safe_col(df_motivos_src, "categoria")) \
    .withColumn("criticidade_raw", safe_col(df_motivos_src, "criticidade"))

df_motivos = df_motivos.withColumn(
    "categoria",
    when(
        (lower(col("nome_motivo")).like("%fatura%")) | 
        (lower(col("nome_motivo")).like("%limite%")) |
        (lower(col("nome_motivo")).like("%contrato%")) |
        (lower(col("nome_motivo")).like("%pagamento%")) |
        (lower(col("nome_motivo")).like("%divida%")) |
        (lower(col("nome_motivo")).like("%renegocia%")), 
        lit("Financeiro")
    )
    .when(
        (lower(col("nome_motivo")).like("%cartao%")) | 
        (lower(col("nome_motivo")).like("%bloqueio%")) |
        (lower(col("nome_motivo")).like("%desbloqueio%")) |
        (lower(col("nome_motivo")).like("%compra%")) |
        (lower(col("nome_motivo")).like("%adicional%")), 
        lit("Cartao")
    )
    .when(
        (lower(col("nome_motivo")).like("%dados%")) | 
        (lower(col("nome_motivo")).like("%cadastra%")) |
        (lower(col("nome_motivo")).like("%telefone%")) |
        (lower(col("nome_motivo")).like("%email%")) |
        (lower(col("nome_motivo")).like("%agencia%")) |
        (lower(col("nome_motivo")).like("%conta%")) | 
        (lower(col("nome_motivo")).like("%endereco%")), 
        lit("Cadastral")
    )
    .when(
        (lower(col("nome_motivo")).like("%app%")) | 
        (lower(col("nome_motivo")).like("%aplicativo%")) |
        (lower(col("nome_motivo")).like("%site%")) |
        (lower(col("nome_motivo")).like("%chatbot%")) |
        (lower(col("nome_motivo")).like("%ura%")) |
        (lower(col("nome_motivo")).like("%problema%")) |
        (lower(col("nome_motivo")).like("%erro%")), 
        lit("Atendimento")
    )
    .when(
        (lower(col("nome_motivo")).like("%ponto%")) | 
        (lower(col("nome_motivo")).like("%beneficio%")) |
        (lower(col("nome_motivo")).like("%programa%")), 
        lit("Beneficios")
    )
    .otherwise(lit("Desconhecida")) 
)
df_motivos = df_motivos.withColumn(
    "id_motivo",
    safe_cast_int(col("id_motivo_raw"))
)

df_motivos = df_motivos.withColumn(
    "criticidade",
    normalize_text(col("criticidade_raw"))
)

df_motivos = df_motivos.withColumn(
    "nome_motivo",
    normalize_text(col("nome_motivo_raw"))
)

df_motivos = df_motivos.withColumn(
    "id_motivo",
    initcap(remove_accents_udf(col("id_motivo")))
)

df_motivos = df_motivos.withColumn(
    "nome_motivo",
    initcap(remove_accents_udf(col("nome_motivo")))
)

df_motivos = df_motivos.withColumn(
    "id_motivo",
    safe_cast_int(col("id_motivo_raw"))
)

final_cols_motivos = ["id_motivo", "nome_motivo", "categoria_detectada", "criticidade"]
df_motivos_final = df_motivos.select(
    col("id_motivo"),
    col("nome_motivo"),
    col("categoria"), 
    col("criticidade"),
    current_timestamp().alias("processed_timestamp")
)
print("Transformações aplicadas.")
df_motivos.display()

## Processamento: dm_canais

Transformação da tabela de canais.
Validação dos canais oficiais (URA, Chatbot, App, Site, Central) e status.

In [0]:
src_table_canais = f"{catalog_name}.{bronze_db_name}.dm_canais"
tgt_table_canais = f"{catalog_name}.{silver_db_name}.dm_canais"

if not safe_table_exists(spark, src_table_canais):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_canais}")

df_canais_src = spark.table(src_table_canais)
total_before_canais = df_canais_src.count()

print(f"Leitura: {src_table_canais}")
print(f"Registros Bronze: {total_before_canais:,}")

df_canais = df_canais_src \
    .withColumn("nome_canal_raw", safe_col(df_canais_src, "nome_canal")) \
    .withColumn("canal_status_raw", safe_col(df_canais_src, "canal_status"))

df_canais = df_canais.withColumn(
    "canal_status",
    normalize_text(col("canal_status_raw"))
)

df_canais = df_canais.withColumn(
    "nome_canal",
    normalize_text(col("nome_canal"))
)

df_canais = df_canais.withColumn(
    "nome_canal",
    initcap(remove_accents_udf(col("nome_canal")))
)

df_canais = df_canais.withColumn(
    "canal_status",
    initcap(remove_accents_udf(col("canal_status")))
)

window_spec = Window.orderBy("nome_canal")

final_cols_canais = ["id_canal", "nome_canal", "canal_status"]

df_canais_final = df_canais.select(
    row_number().over(window_spec).alias("id_canal"),
    col("nome_canal"),
    col("canal_status"),
    current_timestamp().alias("processed_timestamp")
)

print("Transformações aplicadas.")
df_canais_final.display()

## Processamento: ft_chamados

A tabela fato ft_chamados foi enriquecida com dados de tempo (abertura, início e finalização) vindos da tabela auxiliar chamados_hora (silver_table_chamados_hora) através de um LEFT JOIN utilizando as chaves id_chamado e id_cliente.

Colunas de duração (tempo_espera_minutos e tempo_atendimento_minutos) foram recalculadas para garantir a precisão e consistência na camada Silver, substituindo quaisquer valores pré-existentes. O id_atendente nulo foi mantido para a modelagem na camada Gold.

###Processamento da coluna _canal_

A coluna canal foi tratada aplicando:

Realizamos um match com a dimensão dm_canais, utilizando comparação por LIKE/equal normalizados:

**c.canal_norm = dm.nome_canal_norm**


Essa abordagem permitiu recuperar o id_canal oficial da dimensão e associá-lo corretamente aos chamados, mesmo quando o texto chegava sujo, com variações ou codificações incorretas.

In [0]:
path_chamados = f"{catalog_name}.{bronze_db_name}.ft_chamados"
path_motivos = f"{catalog_name}.{bronze_db_name}.ft_motivos"
tgt_table_chamados = f"{catalog_name}.{silver_db_name}.ft_chamados"


if not safe_table_exists(spark, path_chamados):
    raise RuntimeError(f"Tabela fonte não encontrada: {path_chamados}")

df_chamados_src = spark.table(path_chamados)
total_before_chamados = df_chamados_src.count()

print(f"Leitura: {path_chamados}")
print(f"Registros Bronze: {total_before_chamados:,}")

df_chamados = spark.table(path_chamados) 

df_chamados = df_chamados.withColumn(
    "canal_raw", 
    col("canal")
)

col_canal_norm_base = upper(
    remove_accents_udf(
        normalize_text(col("canal_raw"))
    )
)

col_canal_final = (
    when(col_canal_norm_base.like("%ESPECIALIZADO%"), lit("Atendimento Especializado"))
    .when(col_canal_norm_base.like("%INICIAL%"), lit("Atendimento Inicial"))
    .when(col_canal_norm_base.like("U%"), lit("Ura")) 
    .when(col_canal_norm_base.like("%BOT%"), lit("Chatbot"))
    .when(col_canal_norm_base.like("%WEB%"), lit("Web"))
    .when(col_canal_norm_base.like("%mail%"), lit("Email"))
    .otherwise(col_canal_norm_base)
)

df_chamados = (
    df_chamados
    .drop("canal") 
    .withColumn("canal", col_canal_final) 
)

### Processamento da coluna _resolvido_

A coluna resolvido apresentava valores irregulares como:

- Nã­¿½o no lugar de Não

Foi aplicada padronização com base em regras de equivalência:

In [0]:
df_chamados = df_chamados.withColumn(
    "resolvido_raw", 
    col("resolvido")
)

col_resolvido_norm_base = upper(
    remove_accents_udf(
        normalize_text(col("resolvido_raw"))
    )
)

col_resolvido_final = (
    when(col_resolvido_norm_base.like("S%"), lit("Sim"))
    .when(col_resolvido_norm_base.like("N%"), lit("Não"))
    .otherwise(col_resolvido_norm_base)
)

df_chamados = (
    df_chamados
    .drop("resolvido") 
    .withColumn("resolvido", col_resolvido_final) 
)


### Processamento da coluna _motivo_ 

O campo motivo foi o mais complexo devido à forte corrupção de caracteres (ex.: “Contestaï¿½ï¿½o”).
Após a normalização completa por:

- remoção de acentos

- remoção de caracteres inválidos

- substituição de espaços

- caixa alta

Ainda assim, o texto continuava diferente dos motivos corretos existentes na dimensão ft_motivos.

Para resolver isso, aplicamos um processo de similaridade textual com Levenshtein, comparando:

`motivo_norm (do chamado) vs nome_motivo_norm (da dimensão)`


A fórmula de similaridade foi:

`similaridade = (1 - distancia / tamanho_maior) * 100`


Em seguida, selecionamos a melhor correspondência (maior similaridade) por chamada usando janela com row_number().

Esse processo garantiu mapeamento robusto mesmo com textos severamente quebrados, recuperando corretamente valores como:

- “Contestaï¿½ï¿½o” → “Contestação de Fatura”

- “Consulta de limite” → “Consulta de Limite”

E preenchendo o campo id_motivo com a chave da dimensão.


###Processamento dos Timestamps + Cálculo de Tempo

Os campos de datas apresentavam diferentes formatos e casos especiais, como:

- valores não parseáveis

- strings inválidas

- "igual a hora de abertura"

- timestamps existentes em outra tabela Silver

Aplicamos as regras:

1. Substituição de casos especiais

Para início do atendimento:

`when(hora_inicio == "igual a hora de abertura", hora_abertura)`

2. Prioridade para timestamps já corrigidos

Usamos a Silver anterior (tabela h) quando disponível:

`finalizacao = coalesce(h.finalizacao_corrigida, f.finalizacao_original)`

3. Cálculo dos tempos

Tempo de espera:

`(hora_inicio - hora_abertura) / 60`


Tempo de atendimento:

`(hora_finalizacao - hora_inicio) / 60`


Ambos arredondados com round(2).

Esse processo garantiu métricas confiáveis mesmo em casos onde:

- horários estavam corrompidos

- o cliente abriu e iniciou atendimento ao mesmo tempo

- timestamps estavam em formatos diferentes


In [0]:

df_motivos = spark.table(path_motivos) 

df_motivos = df_motivos.withColumn(
    "nome_motivo_norm",
    upper(remove_accents_udf(normalize_text(col("nome_motivo"))))
)

df_chamados = df_chamados.withColumn(
    "motivo_norm",
    upper(remove_accents_udf(normalize_text(col("motivo"))))
)

df_cross = df_chamados.alias("c").crossJoin(df_motivos.alias("m"))

df_cross = df_cross.withColumn(
    "similarity",
    (1 - (levenshtein(col("motivo_norm"), col("nome_motivo_norm")) / F.greatest(length(col("motivo_norm")), length(col("nome_motivo_norm"))))) * 100
)

window = Window.partitionBy("c.id_chamado").orderBy(col("similarity").desc())

df_best = df_cross.withColumn("rank", F.row_number().over(window)).filter(col("rank") == 1)

df_final_clean = df_best.join(
    df_canais_final,
    df_best.canal == df_canais_final.nome_canal,
    "left"
)

join_keys = ["id_chamado", "id_cliente"] 
df_final_clean = (
    df_final_clean.alias("f")
    .join(
        silver_table_chamados.alias("h"), 
        join_keys, 
        "left"
    )
)

df_final_clean = df_final_clean.select(
    col("id_chamado"),
    col("id_cliente"),
    col("id_motivo"),
    col("nome_motivo").alias("motivo"), 
    col("id_canal"),
    col("canal"),
    col("resolvido"),

    coalesce(col("h.hora_abertura_chamado_brasilia"), col("f.hora_abertura_chamado")).alias("hora_abertura_chamado"),
    when(
        col("f.hora_inicio_atendimento") == "igual a hora de abertura",
        coalesce(col("h.hora_abertura_chamado_brasilia"), col("f.hora_abertura_chamado")) # Usa o valor da Abertura
    ).otherwise(
        coalesce(col("h.hora_inicio_atendimento_brasilia"), col("f.hora_inicio_atendimento"))
    ).alias("hora_inicio_atendimento"),
    
    coalesce(col("h.hora_finalizacao_atendimento_brasilia"), col("f.hora_finalizacao_atendimento")).alias("hora_finalizacao_atendimento"),

    col("tempo_espera"),
    col("tempo_atendimento"),
    col("id_atendente"),
    current_timestamp().alias("processed_timestamp")
)


df_final_clean = df_final_clean.withColumn(
    "tempo_espera",
    when(
        (col("hora_inicio_atendimento").isNotNull()) & 
        (col("hora_abertura_chamado").isNotNull()),
        round(
            (unix_timestamp(col("hora_inicio_atendimento")) - 
             unix_timestamp(col("hora_abertura_chamado"))) / 60.0, 
            2
        )
    ).otherwise(lit(None))
)

df_final_clean = df_final_clean.withColumn(
    "tempo_atendimento",
    when(
        (col("hora_finalizacao_atendimento").isNotNull()) & 
        (col("hora_inicio_atendimento").isNotNull()),
        round(
            (unix_timestamp(col("hora_finalizacao_atendimento")) - 
             unix_timestamp(col("hora_inicio_atendimento"))) / 60.0, 
            2
        )
    ).otherwise(lit(None))
)

df_final_clean = df_final_clean.withColumnRenamed("tempo_espera", "tempo_espera_minutos")
df_final_clean = df_final_clean.withColumnRenamed("tempo_atendimento", "tempo_atendimento_minutos")

df_final_clean.orderBy("id_chamado").display()
print(f"Total de registros: {df_final_clean.count():,}")


##Gravação na Camada Silver
Persistência da tabela transformada em formato Delta na camada Silver. Modo overwrite com overwriteSchema para garantir schema atualizado.

In [0]:
df_final_clean.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_chamados)

silver_table_chamados = spark.table(tgt_table_chamados)
final_count_chamados = silver_table_chamados.count()
duplicates_chamados = silver_table_chamados.groupBy("id_chamado").count().filter(col("count") > 1).count()

print(f"Salvo em: {tgt_table_chamados}")
print(f"Contagem final: {final_count_chamados:,}")
print(f"Duplicatas na Silver: {duplicates_chamados}")

display(silver_table_chamados.limit(10))